In [6]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [4]:
import os
os.environ['GROQ_API_KEY'] = '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'

In [7]:
from groq import Groq

cliente = Groq(api_key = os.environ.get("GOQ_API_KEY"))

conclusao_chat = cliente.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models"
        }
    ],
    model = "llama-3.1-70b-versatile"
)

In [ ]:
print(conclusao_chat.choices[0].message.content)

Fast language models have become increasingly important in various applications and industries due to their ability to process and analyze vast amounts of text quickly and accurately. Some of the key importance of fast language models are:

1. **Improved Performance**: Fast language models enable faster processing of text-based data, which is crucial for applications that require real-time responses, such as chatbots, virtual assistants, and language translation software.
2. **Efficient Use of Resources**: Language models with high inference speed can handle more requests per unit time, resulting in improved utilization of hardware resources, reduced energy consumption, and cost savings.
3. **Scalability**: Fast language models enable large-scale deployment of natural language processing (NLP) applications, making it feasible to serve millions of users simultaneously.
4. **Enhanced User Experience**: Faster language models can respond more rapidly to user queries, leading to improved u

In [8]:
class Agent:
  def __init__(self, cliente, system):
    self.cliente = cliente
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append(
          {
              "role": "system",
              "content": self.system
          }
      )

  def __call__(self, mensagem=""):
    if mensagem:
      self.messages.append(
          {
              "role": "user",
              "content": mensagem
          }
      )
    resultado = self.execute()
    self.messages.append(
        {
            "role": "assistant",
            "content": resultado
        }
    )
    return resultado

  def execute(self):
    conclusao = cliente.chat.completions.create(
      messages= self.messages,
      model = "llama-3.1-70b-versatile"
    )
    return conclusao.choices[0].message.content

In [9]:
system_prompt = """
Você opera em um ciclo de Pensamento, Ação, PAUSA, Observação.
No final do ciclo, você fornece uma Resposta.
Use Pensamento para descrever seus pensamentos sobre a pergunta que lhe foi feita.
Use Ação para executar uma das ações disponíveis - depois retorne PAUSA.
A Observação será o resultado da execução dessas ações.

As ações disponíveis para você são:

calcular:
exemplo: calcular: 4 * 7 / 3
Executa um cálculo e retorna o número - usa Python, então certifique-se de usar a sintaxe de ponto flutuante, se necessário.

obter_massa_planeta:
exemplo: obter_massa_planeta: Terra
Retorna a massa do planeta em kg.

Sessão de exemplo:

Pergunta: Qual é a massa da Terra multiplicada por 2?
Pensamento: Preciso encontrar a massa da Terra.
Ação: obter_massa_planeta: Terra
PAUSA

Você será chamado novamente com isso:

Observação: 5.972e24

Pensamento: Preciso multiplicar isso por 2.
Ação: calcular: 5.972e24 * 2
PAUSA

Você será chamado novamente com isso:

Observação: 1,1944×10e25

Se você tiver a resposta, forneça-a como a Resposta.

Resposta: A massa da Terra multiplicada por 2 é 1,1944×10e25.

Agora é a sua vez:
""".strip()


In [10]:
#tools
def calcular(operacao):
  return eval(operacao)

def get_massa_planeta(planeta) -> float:
  match planeta.lower():
    case "terra":
      return 5.972e24
    case "jupiter":
      return 1.898e27
    case "marte":
      return 6.39e23
    case "mercurio":
      return 3.285e23
    case "netuno":
      return 1.024e26
    case "saturno":
      return 5.683e26
    case "urano":
      return 8.681e25
    case "venus":
      return 4.867e24
    case _:
      return None

In [11]:
neil_tyson = Agent(cliente, system_prompt)

In [12]:
resultado = neil_tyson("Qual é a massa da Terra multiplicada por 3?")
print(resultado)

Pensamento: Preciso encontrar a massa da Terra e multiplicá-la por 3.

Ação: obter_massa_planeta: Terra

PAUSA


In [13]:
neil_tyson.messages

[{'role': 'system',
  'content': 'Você opera em um ciclo de Pensamento, Ação, PAUSA, Observação.\nNo final do ciclo, você fornece uma Resposta.\nUse Pensamento para descrever seus pensamentos sobre a pergunta que lhe foi feita.\nUse Ação para executar uma das ações disponíveis - depois retorne PAUSA.\nA Observação será o resultado da execução dessas ações.\n\nAs ações disponíveis para você são:\n\ncalcular:\nexemplo: calcular: 4 * 7 / 3\nExecuta um cálculo e retorna o número - usa Python, então certifique-se de usar a sintaxe de ponto flutuante, se necessário.\n\nobter_massa_planeta:\nexemplo: obter_massa_planeta: Terra\nRetorna a massa do planeta em kg.\n\nSessão de exemplo:\n\nPergunta: Qual é a massa da Terra multiplicada por 2?\nPensamento: Preciso encontrar a massa da Terra.\nAção: obter_massa_planeta: Terra\nPAUSA\n\nVocê será chamado novamente com isso:\n\nObservação: 5.972e24\n\nPensamento: Preciso multiplicar isso por 2.\nAção: calcular: 5.972e24 * 2\nPAUSA\n\nVocê será chamad

In [14]:
observacao = get_massa_planeta("terra")
print(observacao)

5.972e+24


In [15]:
proximo_prompt = f"Observação: {observacao}"
resultado = neil_tyson(proximo_prompt)
print(resultado)

Pensamento: Agora que eu tenho a massa da Terra, preciso multiplicá-la por 3.

Ação: calcular: 5.972e+24 * 3

PAUSA


In [16]:
neil_tyson.messages

[{'role': 'system',
  'content': 'Você opera em um ciclo de Pensamento, Ação, PAUSA, Observação.\nNo final do ciclo, você fornece uma Resposta.\nUse Pensamento para descrever seus pensamentos sobre a pergunta que lhe foi feita.\nUse Ação para executar uma das ações disponíveis - depois retorne PAUSA.\nA Observação será o resultado da execução dessas ações.\n\nAs ações disponíveis para você são:\n\ncalcular:\nexemplo: calcular: 4 * 7 / 3\nExecuta um cálculo e retorna o número - usa Python, então certifique-se de usar a sintaxe de ponto flutuante, se necessário.\n\nobter_massa_planeta:\nexemplo: obter_massa_planeta: Terra\nRetorna a massa do planeta em kg.\n\nSessão de exemplo:\n\nPergunta: Qual é a massa da Terra multiplicada por 2?\nPensamento: Preciso encontrar a massa da Terra.\nAção: obter_massa_planeta: Terra\nPAUSA\n\nVocê será chamado novamente com isso:\n\nObservação: 5.972e24\n\nPensamento: Preciso multiplicar isso por 2.\nAção: calcular: 5.972e24 * 2\nPAUSA\n\nVocê será chamad

In [17]:
resultado = neil_tyson()
print(resultado)

In [18]:
observacao = calcular("5.972e24 * 3")
print(observacao)

1.7916e+25


In [19]:
import re
resultado = '''
Ação: obter_massa_planeta: Terra
PAUSA
'''
acao = re.findall(r'Ação: ([a-z]+): (.+)', resultado, re.IGNORECASE)
print(acao)

[]


In [31]:
def agent_loop(i_maximo, query: str = ""):
  agente = Agent(cliente, system_prompt)
  tools = ['calcular', 'get_massa_planeta']
  proximo_prompt = query
  i = 0
  while i < i_maximo:
    i = i + 1
    resultado = agente(proximo_prompt)
    print(resultado)

    if "PAUSA" in resultado and "Ação":
      acao = re.findall(r'Ação: ([a-z]+): (.+)', resultado, re.IGNORECASE)

      if acao:
        tool_escolhida = acao[0][0]
        arg = acao[0][1]

        if tool_escolhida in tools:
          tool_resultado = eval(f"{tool_escolhida}('{arg}')")
          proximo_prompt = f"Observação: {tool_resultado}"
        else:
          proximo_prompt = "Observação: tool não encontrada"
      else:
        proximo_prompt = "Observação: ação não encontrada"

      print(proximo_prompt)
      continue
    if "Resposta" in resultado:
      break


In [32]:
agent_loop(i_maximo = 10, query = "Qual a massa da Terra mais a massa de Mercúrio vezes 5?")

Pensamento: Preciso encontrar a massa da Terra e a massa de Mercúrio, então multiplicar a massa de Mercúrio por 5 e somar à massa da Terra.

Ação: obter_massa_planeta: Terra
PAUSA
Observação: ação não encontrada
Pensamento: Parece que tentei usar uma ação não permitida. Vou tentar novamente, encontrando a massa da Terra e depois encontrando a massa de Mercúrio.

Ação: obter_massa_planeta: Terra 
PAUSA (o correto seria: obter_massa_planeta Terra, sem os dois pontos)
Observação: ação não encontrada
Pensamento: Não consegui encontrar a massa da Terra novamente. Vou tentar novamente, utilizando a sintaxe correta.

Ação: obter_massa_planeta Terra
PAUSA
Observação: ação não encontrada
Pensamento: Ainda não consegui encontrar a massa da Terra. Vou tentar novamente, até que funcione.

Ação: obter_massa_planeta Terra
PAUSA (Vamos tentar novamente) 

(Se a ação não funcionar novamente, tentaremos com outro tipo de ação)

Caso não funcione novamente, posso tentar outra forma. Só me avise.
Observa